################################################
# NE PAS UTILISER .COLLECT()
#################################################

In [1]:
# import SparkContext and SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import explode
from pyspark import StorageLevel


In [2]:
# create SparkSession with 8gb memory
spark = SparkSession.builder.master("local[8]").appName('loading').config('spark.driver.memory', '8g').getOrCreate()


In [3]:
df = spark.read.options(inferSchema=True, header=False).csv("data\exemple.csv")
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')


In [4]:
# define the function to be applied to each row
def eval_list(x):
    return eval(x)


In [5]:
# define the udf
eval_list_udf = udf(eval_list, ArrayType(StringType()))
# apply the udf to the column
df = df.withColumn('viewers', eval_list_udf(col('viewers')))


In [6]:
df.printSchema()

root
 |-- viewers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- streamer: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- viewership: integer (nullable = true)
 |-- language: string (nullable = true)



In [7]:
df = df.withColumn('viewers', explode(df.viewers))


In [8]:
df = df.groupBy(["viewers", "streamer"]).count()#.persist(StorageLevel.DISK_ONLY)


In [9]:
# filter out the rows with less than 20 count
# df = df.filter(df['count'] > 20)

In [10]:
# print((df.count(), len(df.columns)))

In [11]:
# print the first 10 rows
# df.show(10)

In [12]:
# apply all transformations and store the result in memory
# df.persist(StorageLevel.MEMORY_ONLY)

In [13]:
# write to csv
# df.repartition(8).write.csv("data\lol", mode='overwrite')
df.write.csv("data\lol", mode='overwrite')
# df.toPandas().to_csv('pysparkpuelamortsicamarche.csv')

In [3]:
df = spark.read.csv([f'data\World2022-04-{i}.csv' for i in range(1, 2)], header=False, inferSchema=True)
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')


In [5]:
# drop four last columns
df = df.drop('viewership', 'language', 'date', 'category')

In [6]:
df.select('streamer').show(5)

+---------------+
|       streamer|
+---------------+
|          Emiru|
|       ROSHTEIN|
|       HasanAbi|
|TSM_ImperialHal|
|       alanzoka|
+---------------+
only showing top 5 rows



In [7]:
# print shape of the dataframe
print((df.count(), len(df.columns)))

(9597, 2)


In [13]:
# print the third item in the list of the first row (column viewers)
print(df.select('viewers').first()[0][2])


0025


In [14]:
# run map reduce on the column viewers chunk by chunk
from pyspark.sql.functions import explode


In [15]:
# explode the column viewers with
df = df.withColumn('viewers', explode(df.viewers))


In [16]:
df.show(5)

+-------------+--------+
|      viewers|streamer|
+-------------+--------+
|0000moonlight|   Emiru|
| 001133557799|   Emiru|
|         0025|   Emiru|
|        003ny|   Emiru|
|    007dontae|   Emiru|
+-------------+--------+
only showing top 5 rows



In [17]:
print((df.count(), len(df.columns)))

Py4JJavaError: An error occurred while calling o75.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 9.0 failed 1 times, most recent failure: Lost task 3.0 in stage 9.0 (TID 25) (192.168.2.179 executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.scala:307)
	at scala.collection.Iterator.foreach(Iterator.scala:943)


In [11]:
# add column filled with 1
from pyspark.sql.functions import lit


In [16]:
df = df.withColumn('count', lit(1))

In [13]:
# group viewers by streamer and sum the count column
df = df.groupBy('streamer').sum('count')

In [14]:
# df.show(5) # not working for some reason

In [15]:
# write to csv


Py4JJavaError: An error occurred while calling o70.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\hadoop\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache

In [9]:
# run map reduce on the column viewers
df.select('viewers').rdd.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).take(5)
# explain the above code
# df.select('viewers').rdd.flatMap(lambda x: x) # flatMap the list of viewers to a list of viewers
# .map(lambda x: (x, 1)) # map each viewer to a tuple (viewer, 1)
# .reduceByKey(lambda x, y: x + y) # reduce the tuples by key (viewer) and sum the values
# .take(5) # take the first 5 items


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 3.0 failed 1 times, most recent failure: Lost task 1.0 in stage 3.0 (TID 43) (192.168.2.179 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\valen\AppData\Local\Programs\Python\Python310\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 686, in main
  File "C:\Users\valen\AppData\Local\Programs\Python\Python310\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 676, in process
  File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\Users\valen\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\rdd.py", line 3472, in pipeline_func
    return func(split, p